# Import Dependencies

In [10]:
import numpy as np
from math import *
from scipy.integrate import quad
from scipy.optimize import root

# Fourth-order Runge-Kutta Method
## Calculate Constants

In [2]:
EPSILON = 0.000001
def calculate_kappa(theta, R):
     return sin(theta)**2/R

def calculate_tau(theta, R):
     return sin(theta)*cos(theta)/R

def calculate_f_T(r, theta, s):
    rou = 1
    U0 = 1
    v = 1 #?
    a0=1
    return 2*pi*r*60*rou*U0*cos(theta)**4*(s**10*v/(a0**2+v**2)**6-s**12*v/(a0**2+v**2)**7)

## Define RK Method

In [27]:
def RK4(F, s, y, h):
    k1 = h * np.array(F(s, y))
    k2 = h * np.array(F(s + h/2, y + k1/2))
    k3 = h * np.array(F(s + h/2, y + k2/2))
    k4 = h * np.array(F(s + h, y + k3))
    return y + (k1 + 2*k2 + 2*k3 + k4)/6

## Define & Solve ODE

### Definition

In [30]:
# Define ODE
def F(s, y):
    F_T, F_N, F_B = y
    theta = 0
    R = 1
    r = 1
    kappa = calculate_kappa(theta, R)
    tau = calculate_tau(theta, R)
    f_T = calculate_f_T(r, theta, s)
    f_N = 0.0
    f_B = 0.0
    dF_Tds = F_N * kappa - f_T
    dF_Nds = F_B * tau - F_T * kappa - f_N
    dF_Bds = -F_N * tau - f_B
    return [dF_Tds, dF_Nds, dF_Bds]

In [31]:
# Define Solve
def solve(F, F_T0, F_N0, F_B0, s_start, s_end, h):
    s = s_start
    F_T = F_T0
    F_N = F_N0
    F_B = F_B0
    while s < s_end:
        y = [F_T, F_N, F_B]
        y = RK4(F, s, y, h)
        F_T, F_N, F_B = y
        s += h
    return F_T, F_N, F_B

### Solve

In [32]:
#Solve
theta = 0
R = 1
F_T0 = 0
F_N0 = 0
F_B0 = 0
s_start = 0.0
s_end = 1.0
h = 0.01

In [33]:
F_T, F_N, F_B = solve(F, F_T0, F_N0, F_B0, s_start, s_end, h)
print("F_T(s_end) =", F_T)
print("F_N(s_end) =", F_N)
print("F_B(s_end) =", F_B)

F_T(s_end) = -0.3089415864807307
F_N(s_end) = 0.0
F_B(s_end) = 0.0


## Test RK

### Single Equation 

In [37]:
# Define ODE
def F_test(s, y):
    return -y+1

# Define Solve
def solve_test(F, y0, s_start, s_end, h):
    s = s_start
    y = y0
    while s < s_end:
        y = y
        y = RK4(F, s, y, h)
        y = y
        s += h
    return y

In [40]:
# Solve
y0 = 0.0
s0 = 0.0

s1 = 0.1
h = 0.001
y1 = solve_test(F_test,y0,s0,s1,h)
print("y(0.1) = ", y1)

y(0.1) =  0.09516258196403962


### Equation Set

In [45]:
# Define ODE
def F_test_2(s, y):
    u,v = y
    duds = u + 2*v
    dvds = 3*u + 2*v
    return [duds, dvds]
    
# Define Solve
def solve_test_2(F, u0, v0, s_start, s_end, h):
    s = s_start
    u = u0
    v = v0
    while s < s_end:
        y = [u, v]
        y = RK4(F, s, y, h)
        u, v = y
        s += h
    return u, v

In [47]:
# Solve
u0 = 6
v0 = 4
s0 = 0

s1 = 0.2
h = 0.01
u, v = solve_test_2(F_test_2, u0, v0, s0, s1, h)
print("u(0.2) = ", u)
print("v(0.2) = ", v)

u(0.2) =  10.539625073201622
v(0.2) =  11.715783844343727


# Calculate COD

## Set Constants

In [46]:
x0=10e-9
n=10e-9
m=1860e-9
s=min(int(m/n),x.shape[0])
x = np.arange(x0 - n/2, m - n/2, n)
E=13.5e9
K=1e6
t=30e6
V=0.4
R=14e-9
a=(8*V*t*n)/(pi*E*R)
p=1e-11
l=1.5e-6
b=8*K/(E*sqrt(2*pi))
c=4*V*l*t/(pi*E*R)

## Calculate δ(xi)

In [47]:
# Set up matrixs.
Y = np.ndarray(shape=(s, s))
z = np.ndarray(shape=(s,))
U = np.ndarray(shape=(s,))

# Consider the calculation of δ(xi) as a problem of Yx=z.
# Calculate Y.
for i in range(s):
    for j in range(s):
        if j != i:
            Y[i,j] = a * (2 * np.sqrt(m - x[i]) / np.sqrt(m - x[j]) - np.log((np.sqrt(m - x[i]) + np.sqrt(m - x[j])) / np.abs(np.sqrt(m - x[i]) - np.sqrt(m - x[j]))))
        else:
            Y[j,j] = 1 - a * (np.log((np.sqrt(m - x[j]) + np.sqrt(m - x[j] - p)) / np.abs(np.sqrt(m - x[j]) - np.sqrt(m - x[j] - p))) - 2)

# Calculate z.
for i in range(s):
    z[i] = b * np.sqrt(m - x[i]) + c * (2 * np.sqrt(m * (m - x[i])) - x[i] * np.log((np.sqrt(m) + np.sqrt(m - x[i])) / (np.sqrt(m) - np.sqrt(m - x[i]))))

print(f'The shape of Y is {Y.shape}\nThe shape of z is {z.shape}')

The shape of Y is (185, 185)
The shape of z is (185,)


In [48]:
# Use inverse matrix to solve the linear equations.
z = z.T
D = np.linalg.inv(y)@z

print(f'D represents δ(xi), with a shape of {D.shape}')

D represents δ(xi), with a shape of (185,)


In [50]:
W = D.T
F = 0
for i in range(s):
    U[i] = W[i]*n/sqrt(m-x[i])
    F+=U[i]
Q=2*sqrt(2/pi)*V*t*(sqrt(m)*l-F)/(R*K)
print(f'Q is the toughening ratio, with a value of {Q}')

Q is the toughening ratio, with a value of 1.7316622288288919
